In [1]:
# %conda install -c conda-forge ta-lib -q
# %pip install -r requirements.txt -q

# import boto3
import data
import glob
import matplotlib.pyplot as plt
import models
import numpy as np
import os
import pandas as pd
import pypickle
import strategy

from data import * 
from feature_engineering import create_features, normalize_features
from io import StringIO
from IPython.display import display
from sklearn.linear_model import LassoCV
from utils import log

2025-02-19 21:48:36.717541: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-19 21:48:36.727630: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739972916.739727   10669 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739972916.743323   10669 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-19 21:48:36.755989: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# Define S3 bucket and file path
bucket_name = 'mscfe-capstone-project-bucket'
file_key = 'filtered_data.csv'

filtered = pd.read_csv('filtered_data.csv').drop(['Unnamed: 0'], axis=1)
display(filtered)

,seriesX,seriesY,correlation,spread_adf_stat,spread_adf_pValue,spread_stationary,series_coint_stat,series_coint_pValue,series_cointegrated
0,GOOG,GOOGL,0.999931,-4.971937,0.000025,True,-4.976339,0.000182,True
1,META,DAXEX_ETF,0.888934,-2.917056,0.043400,True,-3.483091,0.033924,True
2,AMZN,BNBUSDT,0.877059,-3.631203,0.005192,True,-3.545100,0.028601,True
3,AMZN,TRXUSDT,0.814577,-3.869803,0.002269,True,-3.387413,0.043768,True
4,SPY_ETF,AVAXUSDT,0.847292,-3.309927,0.014441,True,-3.698512,0.018406,True
5,SPY_ETF,ETHUSDT,0.862483,-3.428205,0.010034,True,-3.737367,0.016394,True
6,EQQQ_ETF,BNBUSDT,0.886060,-3.816404,0.002744,True,-3.670509,0.019987,True
7,BNBUSDT,ADAUSDT,0.901459,-3.279504,0.015820,True,-3.775562,0.014607,True
8,BNBUSDT,BTCUSDT,0.884604,-2.909961,0.044202,True,-3.393160,0.043116,True
9,BNBUSDT,TRXUSDT,0.933562,-3.656380,0.004770,True,-4.003675,0.007095,True


In [3]:
print('********************')
pairs = filtered['seriesX'].iloc[1:] + ' ' + filtered['seriesY'].iloc[1:]
print('Trading pairs:')
for p in pairs:
    print(p)
print('********************')

unique_tickers = pd.concat([filtered['seriesX'].iloc[1:], filtered['seriesY'].iloc[1:]]).drop_duplicates()
print('Unique tickers:')
for ut in unique_tickers:
    print(ut)
print('********************')

# Preview the source data
file_key = 'df_full.csv'
df_full = pd.read_csv('df_full.csv').drop(['Unnamed: 0'], axis=1)
# obj = s3.get_object(Bucket=bucket_name, Key=file_key)
# df_full = pd.read_csv(StringIO(obj['Body'].read().decode('utf-8'))).drop(['Unnamed: 0'], axis=1)
display(df_full.sample(5))

********************
Trading pairs:
META DAXEX_ETF
AMZN BNBUSDT
AMZN TRXUSDT
SPY_ETF AVAXUSDT
SPY_ETF ETHUSDT
EQQQ_ETF BNBUSDT
BNBUSDT ADAUSDT
BNBUSDT BTCUSDT
BNBUSDT TRXUSDT
BNBUSDT XRPUSDT
********************
Unique tickers:
META
AMZN
SPY_ETF
EQQQ_ETF
BNBUSDT
DAXEX_ETF
TRXUSDT
AVAXUSDT
ETHUSDT
ADAUSDT
BTCUSDT
XRPUSDT
********************


,OpenTime,Open,High,Low,Close,Symbol
231102,2024-11-25 10:40:00,240.8100,241.1900,239.4400,239.5400,SOLUSDT
262066,2024-12-06 04:45:00,1.1540,1.1560,1.1500,1.1508,ADAUSDT
312339,2024-12-23 15:40:00,94748.0000,94841.1900,94724.0000,94777.8700,BTCUSDT
82485,2024-10-04 20:10:00,0.5273,0.5273,0.5269,0.5271,XRPUSDT
323561,2024-12-27 13:10:00,3339.4300,3344.8600,3336.4000,3341.4400,ETHUSDT


In [4]:
df_full.Symbol.unique()

array(['NKY225_ETF', 'AAPL', 'AMZN', 'AVGO', 'BRK.B', 'DJIA_ETF', 'GOOG',
       'GOOGL', 'META', 'MSFT', 'NVDA', 'SPY_ETF', 'TSLA', 'DAXEX_ETF',
       'CAC40_ETF', 'EQQQ_ETF', 'FTSE_ETF', 'ADAUSDT', 'AVAXUSDT',
       'BNBUSDT', 'BTCUSDT', 'DOGEUSDT', 'ETHUSDT', 'SOLUSDT', 'TRXUSDT',
       'USDCUSDT', 'XRPUSDT'], dtype=object)

In [5]:
config = {
    'ema': [8, 21, 55],
    'macd': {
        'fast': 12,
        'slow': 26,
        'signal': 9,
    },
    'rsi': [14],
    'bb': {
        'timeperiod': 20,
        'nbdevup': 2,
        'nbdevdn': 2,
    },
    'atr': {
        'timeperiod': 14,
    },
    'stoch': {
        'fastk_period': 14,
        'slowk_period': 3,
        'slowd_period': 3,
    },
    'cci': {
        'timeperiod': 20,
    },
    'willr': {
        'timeperiod': 14,
    },
}

In [6]:
pairs

1       META DAXEX_ETF
2         AMZN BNBUSDT
3         AMZN TRXUSDT
4     SPY_ETF AVAXUSDT
5      SPY_ETF ETHUSDT
6     EQQQ_ETF BNBUSDT
7      BNBUSDT ADAUSDT
8      BNBUSDT BTCUSDT
9      BNBUSDT TRXUSDT
10     BNBUSDT XRPUSDT
dtype: object

In [7]:
untuned_performance = []
scalers = []
validation_dfs = pd.DataFrame()
s = 2

# for p in pairs:
for p in pairs.iloc[0:1]:
    
    tickerX, tickerY = p.split(' ')
    
    spread_df = create_features(tickerX, tickerY,
                                df_full[df_full['Symbol']==tickerX],
                                df_full[df_full['Symbol']==tickerY],
                                config=config,
                                dropna=True)
    
    spread_df, scaler = normalize_features(spread_df)
    spread_df = spread_df.rename(columns={'Spread':'NormalizedSpread'})
    # Save scaler for test set
    scalers.append({p:scaler})

    # ADD LASSO FEATURE SELECTION

    # Generate trading signals based on spreads
    spread_df.loc[:, 'TradingSignal'] = 'No action'
    spread_df.loc[spread_df['NormalizedSpread'] < -1*s, 'TradingSignal']= f'Buy {tickerX}, sell {tickerY}'
    spread_df.loc[spread_df['NormalizedSpread'] > s, 'TradingSignal'] = f'Buy {tickerY}, sell {tickerX}'

    # Preview the data
    display(spread_df.head())
    # Added .copy() to avoid warnings
    model_df = spread_df.drop(['TradingSignal'], axis=1).copy()

    # Create 5 lags
    # To be replaced with/extended by Technical Analysis features
    for lag in range(1, 6):
        model_df.loc[:, f'NormalizedSpread_Lag{lag}'] = model_df['NormalizedSpread'].shift(lag)
    model_df = model_df.dropna()

    # Perform LASSO feature selection
    X_temp = model_df.drop('NormalizedSpread', axis=1)
    y_temp = model_df['NormalizedSpread']
    lasso = LassoCV(cv=5, random_state=42)
    lasso.fit(X_temp, y_temp)
    # Get selected features based on non-zero coefficients
    selected_features = X_temp.columns[abs(lasso.coef_) > 0].tolist()
    model_df = pd.concat([X_temp[selected_features], y_temp], axis=1)

,META_EMA8,META_EMA21,META_EMA55,META_MACD,META_RSI14,META_BBupper,META_BBmiddle,META_BBlower,META_ATR14,META_StochK14,...,DAXEX_ETF_BBupper,DAXEX_ETF_BBmiddle,DAXEX_ETF_BBlower,DAXEX_ETF_ATR14,DAXEX_ETF_StochK14,DAXEX_ETF_StochD3,DAXEX_ETF_CCI20,DAXEX_ETF_WILLR14,NormalizedSpread,TradingSignal
OpenTime,,,,,,,,,,,,,,,,,,,,,
2024-10-30 12:40:00,-0.112015,-0.252482,-0.531448,1.222947,1.690599,-0.262967,-0.234588,-0.198790,-0.459392,1.314240,...,-0.464160,-0.579344,-0.674906,1.140216,-1.478676,-1.436138,-1.803457,-1.397398,0.274508,No action
2024-10-30 12:45:00,-0.061112,-0.218091,-0.506947,1.347383,1.883822,-0.220207,-0.203277,-0.180259,-0.365947,1.314240,...,-0.454816,-0.602047,-0.726982,1.046290,-1.426290,-1.494198,-1.294698,-1.249917,0.359919,No action
2024-10-30 12:50:00,-0.023652,-0.187712,-0.483682,1.421145,1.809052,-0.183612,-0.175700,-0.162827,-0.559904,1.292751,...,-0.456761,-0.620966,-0.761320,1.149427,-1.277946,-1.425712,-0.917759,-0.954954,0.335966,No action
2024-10-30 12:55:00,-0.003625,-0.163879,-0.462794,1.429781,1.488510,-0.155382,-0.155089,-0.150716,-0.608739,1.297948,...,-0.462336,-0.637660,-0.788027,1.023118,-1.073270,-1.287395,-0.733970,-0.821181,0.281800,No action
2024-10-30 13:00:00,0.019626,-0.139024,-0.441350,1.446421,1.594570,-0.122025,-0.136847,-0.148741,-0.681066,1.261997,...,-0.473791,-0.654131,-0.808836,0.874105,-0.885385,-1.102847,-0.604544,-0.720969,0.318883,No action


/home/alfred/miniconda3/envs/tf_py311/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.806e-01, tolerance: 1.144e-01
  model = cd_fast.enet_coordinate_descent(


In [8]:
(ridge, ridge_mse, ridge_test_df, ridge_time, ridge_memory) = models.ridge_regression(df=model_df, p=p) 

Ridge Regression MSE: 0.0026686735800975766


In [13]:
(ridge, ridge_mse, ridge_test_df, ridge_time, ridge_memory) = models.ridge_regression(df=spread_df.drop(['TradingSignal'], axis=1).copy(), p=p, alpha=0.1) 

Ridge Regression MSE: 0.002728460785870152


In [10]:
(ridge, ridge_mse, ridge_test_df, ridge_time, ridge_memory) = models.ridge_regression(df=model_df, p=p, alpha=0.5) 

Ridge Regression MSE: 0.002610278525837848


In [11]:
(ridge, ridge_mse, ridge_test_df, ridge_time, ridge_memory) = models.ridge_regression(df=model_df, p=p, alpha=2) 

Ridge Regression MSE: 0.002745538893262572


In [ ]:
ridge

In [ ]:
ridge_test_df.PredictedSignal.value_counts()

In [ ]:
spread_df.TradingSignal.value_counts()

In [ ]:
spread_df